In [2]:
import load
import oed

# jsonファイルの読み込み
result = load.load_result_json("../result/Result.json")

gene = result[0]["gene"]

N, M, theta, w_on, w_off, w, g, w_osc, w_nmj = oed.weight(gene)
alpha, x_peak, y_peak, dt, T, f, v, time, tau = oed.constant("setting")
N_, M_, f_inv, T_ = oed.time_constant_step(gene, "setting")

In [ ]:
# connectome
import load
import figure

result = load.load_result_json("../result/Result.json")
file_path_base = "../figure/unlimited/connectome/connectome_"
for i in range(10):
    gene = result[i]["gene"]
    file_path = "{}{}".format(file_path_base, i)
    figure.connectome(gene, file_path)

In [ ]:
# newron_output
import load
import figure

result = load.load_result_json("../result/Result.json")
file_path_base = "../figure/unlimited/newron/newron_"
for i in range(10):
    gene = result[i]["gene"]
    file_path = "{}{}".format(file_path_base, i)
    figure.newron_output(gene, file_path)

In [ ]:
# bearing_vs_turning_bias
import figure

in_file_path_base = "../output/bearing_vs_turning_bias/b_vs_t_"
out_file_path_base = "../figure/unlimited/bearing_vs_turning_bias/b_vs_t_"
for i in range(10):
    in_file_path = "{}{}.txt".format(in_file_path_base, i)
    out_file_path = "{}{}".format(out_file_path_base, i)
    figure.Bearing_vs_Turing_bias(in_file_path, out_file_path)

In [ ]:
# nomal_gradient_vs_turning_bias
import figure

in_file_path_base = "../output/nomal_gradient_vs_turning_bias/n_vs_t_"
out_file_path_base = "../figure/unlimited/nomal_gradient_vs_turning_bias/n_vs_t_"
for i in range(10):
    in_file_path = "{}{}.txt".format(in_file_path_base, i)
    out_file_path = "{}{}".format(out_file_path_base, i)
    figure.Normal_gradient_vs_Turing_bias(in_file_path, out_file_path)

In [ ]:
# translational_gradient_vs_turning_bias
import figure

in_file_path_base = "../output/translational_gradient_vs_turning_bias/t_vs_t_"
out_file_path_base = (
    "../figure/unlimited/translational_gradient_vs_turning_bias/t_vs_t_"
)

for i in range(10):
    in_file_path = "{}{}.txt".format(in_file_path_base, i)
    out_file_path = "{}{}".format(out_file_path_base, i)
    figure.Translational_gradient_vs_Turing_bias(in_file_path, out_file_path)

In [ ]:
# trajectory
import load
import figure

file_path_base = "../figure/unlimited/trajectory/trajectory_"
result = load.load_result_json("../result/Result.json")
for i in range(10):
    gene = result[i]["gene"]
    file_path = "{}{}".format(file_path_base, i)
    figure.trajectory(gene, 10, 7, file_path)

In [ ]:
# trajectory
import load
import figure

file_path_base = (
    "../figure/unlimited/trajectory_membrane_potential/trajectory_membrane_potential_"
)
result = load.load_result_json("../result/Result.json")
for i in range(10):
    gene = result[i]["gene"]
    file_path = "{}{}".format(file_path_base, i)
    figure.trajectory_membrane_potential(gene, 10, file_path)